# Whisper SageMaker fine-tuning scratch

> *This notebook has been tested with the `PyTorch 2.0.0 Python 3.10 (CPU Optimized)` kernel in SageMaker Studio*

## Installs and imports

Long-term, this notebook should work fine with `Data Science 3.0` and not need Hugging Face dependencies (because everything will be done in training jobs)... But for initial experimentation, it's useful to have transformers available locally:

In [ ]:
%pip install "datasets>=2.14,<3" "evaluate>=0.4,<0.5" jiwer librosa transformers==4.28

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import os

# External Dependencies:
import boto3
import datasets
from huggingface_hub import notebook_login as hf_notebook_login
import sagemaker
from sagemaker.huggingface import HuggingFace as HuggingFaceEstimator, HuggingFaceProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Local Dependencies:
from src.sagemaker_whisper import notebook as util


# Connect to AWS SDKs:
smsess = sagemaker.Session()
role = sagemaker.get_execution_role()

# Configure data locations and setup:
bucket_name = smsess.default_bucket()
s3_prefix = "whisper/"
os.makedirs("data/cache", exist_ok=True)
os.environ["HF_DATASETS_CACHE"] = "data/cache"

language = "Indonesian"
lang_code = "id"

## Fetch the source data from Hugging Face Hub

If you try to fetch (even one language subset from) HF Common Voice dataset direct from within a SageMaker Studio notebook, the process will take hours because it unpacks many small files and the underlying user storage in Studio is file-system-based rather than block-store-based.

Therefore use a SageMaker Processing job to initially fetch and un-pack the data to S3:

In [ ]:
# Need to specify explicit image_uri otherwise HuggingFaceProcessor complains about not having a
# non-GPU image when used with m*/c*/etc non-accelerated instance types:
image_uri = sagemaker.image_uris.retrieve(
    "huggingface",
    region=os.environ["AWS_REGION"],
    version="4.28",
    base_framework_version="pytorch2.0",
    py_version="py310",
    image_scope="training",
)

In [ ]:
source_data_s3uri = f"s3://{bucket_name}/{s3_prefix}data/common-voice-{lang_code}"
source_data_s3uri

In [ ]:
# Remove previous version of data if you want:
# !aws s3 rm --quiet {source_data_s3uri}

In [ ]:
%%time

# TODO: Combine with https://huggingface.co/datasets/google/fleurs/viewer/id_id/train as well?

processor = HuggingFaceProcessor(
    base_job_name=f"whisper-fetch-{lang_code}",
    role=role,
    py_version="py310",
    pytorch_version="2.0",
    transformers_version="4.28",
    image_uri=image_uri,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    volume_size_in_gb=40,
)

processor.run(
    code="entry_fetch.py",
    source_dir="src",
    inputs=None,
    outputs=[
        ProcessingOutput(
            source="/opt/ml/processing/output",
            destination=source_data_s3uri,
            output_name="output",
            s3_upload_mode="Continuous",
        ),
    ],

    arguments=[
        # Name/ARN of your AWS Secrets Manager Secret containing your Hugging Face Hub access token:
        "--hf_secret_id", "hfhub",
        "--language_code", lang_code,
        # Normalize audio sample rate at pre-processing time to save time in training job:
        "--norm_sample_rate", "16000",
        # More shards to help scale later pre-processing in the training job:
        "--save_num_shards", "48",
    ],
    wait=True,
    logs=True,
)

In [ ]:
# Optionally download the prepared dataset to notebook:

!rm -rf data/common-voice-{lang_code}
!aws s3 sync --quiet {source_data_s3uri} data/common-voice-{lang_code}

## Model training

In [ ]:
hyperparameters = {
    "model_name_or_path": "openai/whisper-small",  # Not whisper-large-v2 yet: CUDA OOM
    "language": language,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 32,
    "gradient_accumulation_steps": 1,
    "gradient_checkpointing": "true",  # Not needed if not VRAM-constrained?
    "fp16": "true",

    "learning_rate": 5e-6,
    "warmup_steps": 500,
    "max_steps": 3000,
    "evaluation_strategy": "steps",
    "save_strategy": "steps",
    "save_steps": 200,
    "eval_steps": 200,
    "logging_steps": 25,

    "early_stopping_patience": 10,
    "load_best_model_at_end": "true",
    "metric_for_best_model": "wer",
    "greater_is_better": "false",

    # Early stopping implies checkpointing every evaluation, so limit the total checkpoints
    # kept to avoid filling up disk:
    "save_total_limit": 10,
    "seed": 42,

    "predict_with_generate": "true",
    "generation_max_length": 255,
    # "push_to_hub": False,
}

metric_definitions = [
    {"Name": "epoch", "Regex": util.get_hf_metric_regex("epoch")},
    {"Name": "learning_rate", "Regex": util.get_hf_metric_regex("learning_rate")},
    {"Name": "train:loss", "Regex": util.get_hf_metric_regex("loss")},
    {"Name": "validation:loss", "Regex": util.get_hf_metric_regex("eval_loss")},
    {
        "Name": "validation:samples_per_sec",
        "Regex": util.get_hf_metric_regex("eval_samples_per_second"),
    },
    {"Name": "validation:wer", "Regex": util.get_hf_metric_regex("eval_wer")},
]

estimator = HuggingFaceEstimator(
    base_job_name=f"whisper-ft-{lang_code}",
    entry_point="entry_train.py",
    source_dir="src",
    output_path=f"s3://{bucket_name}/{s3_prefix}trainjobs",
    role=role,
    py_version="py310",
    pytorch_version="2.0",
    transformers_version="4.28",
    instance_count=1,
    instance_type="ml.g5.4xlarge",  # E.g. "ml.g5.4xlarge", "ml.p3.2xlarge"
    # volume_size=30,
    keep_alive_period_in_seconds=900,  # Enable warm pools for faster debugging (you need quota)

    environment={
        "PIP_CACHE_DIR": "/opt/ml/sagemaker/warmpoolcache/pip",  # For warm pools
        # "TOKENIZERS_PARALLELISM": "false",
    },
    hyperparameters=hyperparameters,
    metric_definitions=metric_definitions,
)

In [ ]:
estimator.fit({"dataset": source_data_s3uri})

## Donezo